In [1]:
%reset -f
import pandas as pd
import numpy as np
import datetime as dt
import random
import time
import math
import itertools

almNum = 3
ocorr = 100
t_sample = 60
base_hour = dt.datetime(2018, 1,9,9,0,0)
occor_num = 0

#Settings alarms

a = np.zeros(1, dtype=int)
ocorr = 10
duration = 120 #120sec
hour_init = base_hour

b = np.zeros(1, dtype=int)
pAb = 0.1
delay_b = 60 #seconds
duration_b = 120

c = np.zeros(1, dtype=int)
pAc = 0.6
delay_c = 15 #seconds
duration_c = 120


#Alarm Series Generation, A (cause), B(Effect). C(Effect)
while(occor_num < ocorr): 
    
    #A generation - begin
    srtd_hour = random.normalvariate(3, 1)
    srtd_hour_begin = hour_init + dt.timedelta(hours=srtd_hour)
    srtd_hour_end = srtd_hour_begin + dt.timedelta(seconds = duration)

    idx_init_a_occor = int(math.ceil((srtd_hour_begin - base_hour).total_seconds()/t_sample))
    idx_end_a_occor = int(math.ceil((srtd_hour_end - base_hour).total_seconds()/t_sample))
        
    if(idx_end_a_occor > a.size):
            a.resize(idx_end_a_occor)

    for i in np.arange(idx_init_a_occor, idx_end_a_occor + 1):
        a[i-1] = 1
    #A generation - end
         
    
    #B generation begin
    srtd_prob_b = random.uniform(0,1)  
    if srtd_prob_b <= pAb:
        srtd_hour_begin_b = srtd_hour_begin + dt.timedelta(hours = delay_b/3600)
        srtd_hour_end_b = srtd_hour_begin_b + dt.timedelta(seconds=duration_b)

        idx_init_b_occor = int(math.ceil((srtd_hour_begin_b - base_hour).total_seconds()/t_sample))
        idx_end_b_occor = int(math.ceil((srtd_hour_begin_b - base_hour).total_seconds()/t_sample))

        if(idx_end_b_occor > b.size):
                b.resize(idx_end_b_occor)

        for j in np.arange(idx_init_b_occor, idx_end_b_occor +1):
            b[j-1] =  1  
    #B generation end

    #C generation begin
    srtd_prob_c = random.uniform(0,1)
    if srtd_prob_c <= pAc:
        srtd_hour_begin_c = srtd_hour_begin + dt.timedelta(hours = delay_b/3600)
        srtd_hour_end_c = srtd_hour_begin_c + dt.timedelta(seconds=duration_c)

        idx_init_c_occor = int(math.ceil((srtd_hour_begin_c - base_hour).total_seconds()/t_sample))
        idx_end_c_occor = int(math.ceil((srtd_hour_end_c - base_hour).total_seconds()/t_sample))
        
        if(idx_end_c_occor > c.size):
            c.resize(idx_end_c_occor)

        for j in np.arange(idx_init_c_occor, idx_end_c_occor +1):
            c[j-1] = 1    
    #C generation end
    
    hour_init = srtd_hour_begin
    occor_num = occor_num + 1

#Making series the same length
max_len = max(a, b, c, key=len).size
a = np.concatenate([a, np.zeros(max_len - a.size)])
b = np.concatenate([b, np.zeros(max_len - b.size)])
c = np.concatenate([c, np.zeros(max_len - c.size)])
sum(a)

30.0

In [24]:
b = b.astype(int)
a = a.astype(int)

def joint_probability(k,l, h, a, b):
    
    '''
        k B time horizon
        l A time horizon
        h instant in the future of serie B
        
        a, b array type'''

    #Alarm Series A (cause), B (effect), same len
    #tested
    sizeSeries = a.size
    transEntropy = 0
    numStates = 2**(k + l  + 1)
    combinations = list(map(list, itertools.product([0, 1], repeat=k+l+1)))
    counting = np.zeros(numStates)
    prob_cnjt = np.zeros(numStates)
    a_prob_ind = []
    b_prob_ind = []

    #calculo da probabilidade conjunta p(i_sub_t+1), i_sub_t**k, j_sub_t**l)
    inicio = np.max([k,l]) - 1
    for i in np.arange(inicio, sizeSeries - 1):
        for hk in np.arange(0,k):
                b_prob_ind.append(b[i - hk])
        for hl in np.arange(0,l):
                a_prob_ind.append(a[i - hl])

        ab = [b[i + h]] + b_prob_ind + a_prob_ind 
        index_comb = combinations.index(ab)
        counting[index_comb] = counting[index_comb] + 1

        a_prob_ind = []
        b_prob_ind = []

    total = sum(counting)
    for i, cnt in enumerate(counting):
        prob_cnjt[i] = cnt/total
        
    return prob_cnjt

In [27]:
#Joint probability evaluation p(i_t+h, i_t**k)
#tested
states_ith_ik = list(map(list, itertools.product([0, 1], repeat=k + 1)))
p_jnt_ith_ik = np.zeros(2**(k+1))
                         
for i, state in enumerate(states_ith_ik):
    for j, comb in enumerate(combinations):
        if comb[0:k+1] == state:
            p_jnt_ith_ik[i] = p_cjnt_ith_ik[i] + prob_cnjt[j]
            

In [4]:
#Conditional probability p(i_sub_t+h|i**k, j**l) from log2 numerator
#tested

states_i_j = list(map(list, itertools.product([0, 1], repeat=k+l)))

size = int(prob_cnjt.size/2)
conditional_num = np.zeros(2**(k+l+1))

for i,state in enumerate(states_i_j):
    index_zero = combinations.index([0] + state)
    prob_zero = prob_cnjt[index_zero]
   
    index_one = combinations.index([1] + state)
    prob_one = prob_cnjt[index_one]
    
    if(prob_zero + prob_one != 0):
        conditional_num[i] = prob_zero/(prob_zero+ prob_one)
        conditional_num[i + 2**(k+l)] = prob_one/(prob_zero+ prob_one)

In [5]:
#Conditional probability p(i_sub_t+h|i**k) from log2 denominator
#tested

size = int((2**(k+1))/2)
count_zero  = 0
count_um = 0
conditional_den = np.zeros(2**(k+1))

states_i = list(map(list, itertools.product([0, 1], repeat=k)))
states_i_plus_h_i_k = list(map(list, itertools.product([0, 1], repeat=k+1)))


for i, state in enumerate(states_i):
    index_zero = states_i_plus_h_i_k.index([0] + state)
    prob_zero = p_jnt_ith_ik[index_zero]

    index_one = states_i_plus_h_i_k.index([1] + state)
    prob_one = p_jnt_ith_ik[index_one]

    if(prob_zero + prob_one != 0):
        conditional_den[i] = prob_zero/(prob_zero+ prob_one)
        conditional_den[i + 2**(k)] = prob_one/(prob_zero+ prob_one)

In [6]:
conditional_den

array([9.99390244e-01, 1.00000000e+00, 6.09756098e-04, 0.00000000e+00])

In [7]:
#Division of the conditionals in log2 
#tested
conditional_div = np.zeros(conditional_num.size)
states_den = list(map(list, itertools.product([0, 1], repeat=1+k)))
for j, comb in enumerate(combinations):
    if(conditional_den[states_den.index(comb[0:k+1])] != 0):
        conditional_div[j] = conditional_num[j]/conditional_den[states_den.index(comb[0:k+1])]            
   

In [8]:
#Tranfer entropy final evaluation

#log2 from the division of the conditionals -> p(i_sub_t+h|i_sub_t**k, j_sub_t**l) /p(i_sub_t+h|i_t**k)

log2_div_cond = np.log2(conditional_div[conditional_div!=0])
te = np.sum(prob_cnjt[conditional_div!=0]*log2_div_cond)

In [9]:
te

0.003562598515436798

In [10]:
#Cálculo da probabilidade marginal i_sub_t**
# p_marg_i_k = np.zeros(2**k)
# estados_i = list(map(list, itertools.product([0, 1], repeat=k)))
# for i, est_i in enumerate(estados_i):
#     for j, comb in enumerate(combinations):
#         if  comb[1:k+1] == est_i:
#             p_marg_i_k[i] = p_marg_i_k[i] + prob_cnjt[j]
    



In [11]:
# #Cálculo da conjunta p(ik, jl)
# prob_cnjt_ik_jl = []
# size = int(prob_cnjt.size/2)

# for i in np.arange(0,size):
#     prob_cnjt_ik_jl.append(prob_cnjt[i] + prob_cnjt[2**(k+l) + i])
# prob_cnjt_ik_jl = np.asarray(prob_cnjt_ik_jl)